# Exercises from Linear Algebra for DS

In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.io as pio
import scipy

pio.renderers.default='iframe'
#plt.figure(figsize=(12, 12), dpi=80)
plt.rcParams["figure.figsize"] = (7,7)

In [2]:
#function to create nd vectors in (0, 10, 0, 10)
def vec(dim=2):
    return np.random.rand(dim)*10

# function to create row-vectors
def row(dim=2):
    return np.array([vec(dim=2)])

## Chapter 6

### Exercise 6.1

Calculate frobenius norm